<h1><center>Proyecto Interciclo</center></h1>


### Nombre:
Andrés Zeas G.

## Objetivo

Consolidar los conocimientos adquiridos en clase para los métodos de búsqueda y bases de datos orientadas a grafos.

## Enunciado

Diseñe y desarrolle un sistema recopilador que permita obtener las noticias o facebook de los candidatos y almacenar dentro de una base de datos orientados a grafos:

Webscraping es la técnica de extraer datos contenidos en un formato no estructurado en una página web y llevarlos a una estructura fácil de usar.
Es por ello, que se desea crear nuevos métodos que permitan la recopilación masiva de informaciones para su posterior estudio y correlación en forma de big data.
En base a ello, vamos a obtener los datos de lo que esta hablando las noticias de los candidatos dentro del Ecuador y almacenar los datos dentro de una base de datos orientadas a grafos.

## Neo4j + Python
Para instalar la libreria solo tenemos que ejecutar

`pip install neo4j`

Procedemos a verificar que la instalacion este correcta con la siguiente instrucción.

## Ejecucion desde Python
A continuacion configuran la cadena de conexion y verifican que este correctamente instalado la libreria si genera un error de seguridad TTL [4] agregar 'encrypted=False'

In [18]:
from neo4j import GraphDatabase

## Desarrollo

Haremos uso de Beautiful Soup la cual es una biblioteca de Python que utiliza su analizador html / xml preinstalado y convierte la página web / html / xml en un árbol que consta de etiquetas, elementos, atributos y valores. Para ser más exactos, el árbol consta de cuatro tipos de objetos, Tag, NavigableString, BeautifulSoup y Comment. Este árbol se puede "consultar" utilizando los métodos / propiedades del objeto BeautifulSoup que se crea a partir de la biblioteca del analizador.

Procedemos a instalar Beautiful Soup con el siguiente comando: `pip install beautifulsoup4` y lo importamos a nuestro proyecto.

In [19]:
from bs4 import BeautifulSoup
import requests

Generamos el diseño de como van a estar conformados nuestros nodos dentro de la base de Neo4j

In [20]:
class Neo4jService(object):
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
        
    def close(self):
        self._driver.close()      
    
    #Creación de Nodos
    def nodo_cabeceraTitulo(self, tx, nombre):
        tx.run("CREATE (:Titulo {nombre: $nombre})", nombre=nombre)
        
    def nodo_presidente(self, tx, nombre):
        tx.run("CREATE (:Presidente {nombre: $nombre})", nombre=nombre)
        
    def nodo_noticia(self, tx, nombre):
        tx.run("CREATE (:Noticias {nombre: $nombre})", nombre=nombre)
        
    def nodo_fechaNoticia(self, tx, nombre):
        tx.run("CREATE (:Fecha {nombre: $nombre})", nombre=nombre)   
        
    def nodo_contenidoNoticia(self, tx, nombre):
        tx.run("CREATE (:Contenido {nombre: $nombre})", nombre=nombre)   
        
    def nodo_titulosNoticias(self, tx, nombre):
        tx.run("CREATE (:Titulos {nombre: $nombre})", nombre=nombre)
        
    def nodo_fechasNoticias(self, tx, nombre):
        tx.run("CREATE (:Fechas {nombre: $nombre})", nombre=nombre)
     
    #Relaciones
    def relacion_noticia(self, tx, nombre_noticias, nombre_noticia):
        tx.run("MATCH (a:Noticias {nombre: $nombre_noticias}) "
               "MATCH (b:Contenido {nombre: $nombre_noticia}) "
               "MERGE (a)-[:Noticias]->(b)",
               nombre_noticias=nombre_noticias, nombre_noticia=nombre_noticia)
        
    def relacion_titulosCabeceras(self, tx, nombre_titulos, nombre_titulo):
        tx.run("MATCH (a:Titulo {nombre: $nombre_titulos}) "
               "MATCH (b:Titulos {nombre: $nombre_titulo}) "
               "MERGE (a)-[:TituloNoticia]->(b)",
               nombre_titulos=nombre_titulos, nombre_titulo=nombre_titulo)
        
    def relacion_fechaNoticias(self, tx, tiempo_fechas, nombre_fecha):
        tx.run("MATCH (a:Fecha {nombre: $tiempo_fechas}) "
               "MATCH (b:Fechas {nombre: $nombre_fecha}) "
               "MERGE (a)-[:Fecha]->(b)",
               tiempo_fechas=tiempo_fechas, nombre_fecha=nombre_fecha)

    def relacion_presidente_tituloNoticia(self, tx, presidente, titulo_noticia):
        tx.run("MATCH (a:Presidente {nombre: $presidente}) "
               "MATCH (b:Titulo {nombre: $titulo_noticia}) "
               "MERGE (a)-[:Titulos]->(b)",
               presidente=presidente, titulo_noticia=titulo_noticia)
        
    def relacion_presidente_fechaNoticia(self, tx, presidente, fecha_noticia):
        tx.run("MATCH (a:Presidente {nombre: $presidente}) "
               "MATCH (b:Fecha {nombre: $fecha_noticia}) "
               "MERGE (a)-[:Fechas]->(b)",
               presidente=presidente, fecha_noticia=fecha_noticia)
    
    def relacion_presidente_contenidoNoticia(self, tx, presidente, noticia_contenido):
        tx.run("MATCH (a:Presidente {nombre: $presidente}) "
               "MATCH (b:Noticias {nombre: $noticia_contenido}) "
               "MERGE (a)-[:Contenido]->(b)",
               presidente=presidente, noticia_contenido=noticia_contenido)

Se procede a la conección con la base de datos.
Manda a crear los nodos.

In [21]:
neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'admin')
with neo4j._driver.session() as session:
    session.write_transaction(neo4j.nodo_presidente , "Presidente")
    session.write_transaction(neo4j.nodo_cabeceraTitulo , "Titulos")
    session.write_transaction(neo4j.nodo_fechaNoticia , "Fecha")
    session.write_transaction(neo4j.nodo_noticia , "Noticias")

#Enlaces de datos
with open("datos.txt","r") as archivo:
    for enlace in archivo:
        pageGoo= requests.get(enlace)
        soupGoo = BeautifulSoup(pageGoo.content,'html.parser')
        resultaFecha = soupGoo.find_all("span",{"class":"r0bn4c rQMQod"})
        resultaTitulo = soupGoo.find_all("div",{"class":"BNeawe vvjwJb AP7Wnd"})
        resultaTexto = soupGoo.find_all("div",{"class":"kCrYT"}) 
        contenidoTextoGoogle=list()
        
        for resu in resultaTexto:
            contenidoTextoGoogle.append(resu.text)
        auxGo=list(set(contenidoTextoGoogle)) 
        
        for ll in auxGo:
            session.write_transaction(neo4j.nodo_contenidoNoticia,ll)
            session.write_transaction(neo4j.relacion_noticia,"Noticias",ll)
        tituloGoo=list()
        
        for i in resultaTitulo:
            tituloGoo.append(i.text) 
            
        for titulo in tituloGoo:
            session.write_transaction(neo4j.nodo_titulosNoticias,titulo)
            session.write_transaction(neo4j.relacion_titulosCabeceras,"Titulos",titulo)  
            print('TITULO')
            print(titulo)
        enlaceFecha=list()
        
        for e2 in resultaFecha:
            enlaceFecha.append(e2.text)
            
        for fecha in enlaceFecha:
            print('FECHA')
            print(fecha)
            session.write_transaction(neo4j.nodo_fechasNoticias,fecha)
            session.write_transaction(neo4j.relacion_fechaNoticias,"Fecha",fecha) 
            
session.write_transaction(neo4j.relacion_presidente_tituloNoticia,"Presidente","Titulos")
session.write_transaction(neo4j.relacion_presidente_contenidoNoticia,"Presidente","Noticias")
session.write_transaction(neo4j.relacion_presidente_fechaNoticia,"Presidente", "Fecha")
print("Todos los nodos han sido generados!!")

TITULO
Presidente Guillermo Lasso: Nunca cederemos a amenazas porque eso echaría abajo el buen trabajo que estamos haciendo
TITULO
'Los quiteños no merecen lo que hoy sucede en el Municipio', dice ...
TITULO
El 73,5% de la población aprueba la gestión del presidente ...
TITULO
Militares entregaron el bastón de mando a Guillermo Lasso
TITULO
Aprobación al presidente Guillermo Lasso escala al 73,5 %, y al 78,4 % por su forma de gobernar, según Cedatos
TITULO
Roberto Salas se suma al gabinete de Guillermo Lasso para promover alianzas público-privadas
TITULO
Pascual del Cioppo: ‘Malentendido’ entre Guillermo Lasso y Jaime Nebot debe superarse por el bien del país
TITULO
Leonidas Iza: Guillermo Lasso respondió con prepotencia, cerrando todo diálogo
TITULO
El presidente Guillermo Lasso firmó un decreto para simplificar ...
TITULO
Guillermo Lasso presentó el Plan Fénix de vacunación; habló de las ...
FECHA
hace 1 hora
FECHA
 · 
FECHA
hace 3 horas
FECHA
 · 
FECHA
hace 1 día
FECHA
 · 
FECHA
hac

FECHA
Hace 3 semanas
FECHA
 · 
FECHA
Hace 4 semanas
FECHA
 · 
FECHA
Hace 2 semanas
FECHA
 · 
FECHA
hace 1 día
FECHA
 · 
FECHA
Hace 2 semanas
FECHA
 · 
FECHA
hace 3 días
FECHA
 · 
FECHA
Hace 3 semanas
FECHA
 · 
FECHA
Hace 3 semanas
FECHA
 · 
TITULO
Lasso preside reunión contra desnutrición crónica infantil
TITULO
Human Rights Watch insta a la Asamblea y a Guillermo Lasso a ...
TITULO
Quiénes son y qué hacen los seis consejeros del presidente Lasso
TITULO
El Gobierno pretende eliminar el hacinamiento en las prisiones, reconociendo derechos carcelarios a 5.000 reos
TITULO
Productores bananeros anuncian movilizaciones y paralización de ...
TITULO
El 68% del gabinete de Lasso viene de fuera de sus 'canteras'
TITULO
Presidente Lasso y canciller Montalvo encabezan comitiva que ...
TITULO
Presidente Guillermo Lasso busca acuerdo de libre comercio con ...
TITULO
Universidades volverán a ofertar posgrados autofinanciados en el ...
TITULO
Lasso retomó actividad en TikTok, tras críticas por video 

## Conclusiones y recomendaciones

* Investigar el uso de las diferentes herramientas que permiten web scraping de acuerdo al tipo de información que se requiera obtener.
* Este método de etracción de  contenido nos resulta bastante util para realizar analisis de datos a grandes escalas.
* Las bases de datos Neo4j debido a su versatilidad nos permite manipular la información a nuestra conveniencia para relizar diferentes tipso de analisis, asi como los algoritmos predefinidos que nos permiten un analisis más profundo de la información.

## Bibliografía y referencias

* Beaufiful Soap
https://riptutorial.com/es/beautifulsoup#:~:text=Beautiful%20Soup%20es%20una%20biblioteca,%2C%20NavigableString%2C%20BeautifulSoup%20y%20Comment.

* Neo4j + Python
https://neo4j.com/developer/python/

